In [ ]:
! pip install langgraph langsmith

In [ ]:
from utility.llm_factory import LLMFactory
llm = LLMFactory.get_llm('groq')

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x77e0038b82d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x77e0038d1890>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=512)

### Start Building Chatbot using LangGraph

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
class State(TypedDict):
    # Messages have the type "list". The "add_messages" function
    # in the annotation defined how this state key is updated.
    # in this case, it appends messages to the list, rather than overwriting it.
    messages: Annotated[list, add_messages]


In [33]:
graph_builder = StateGraph(State)

def chatbot(state: State):
    return {"messages": llm.invoke(state['messages'])}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

In [35]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting the chatbot. Goodbye!")
        break

    for event in graph.stream({'messages': ("user", user_input)}):
        print(event.values())
        for value in event.values():
            print(value['messages'])
            print("Assistant:", value['messages'].content)

dict_values([{'messages': AIMessage(content='Please provide me with a question or topic so I can assist you!  \n\nFor example, you could ask me:\n\n* **A question:** "What is the capital of France?"\n* **A creative writing prompt:** "Write a short story about a robot who learns to feel emotions."\n* **A request for information:** "Tell me about the history of the internet."\n* **A task:** "Summarize this article for me."\n\n\nI\'m here to help! 😊\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 11, 'total_tokens': 115, 'completion_time': 0.189090909, 'prompt_time': 0.001941687, 'queue_time': 0.073960812, 'total_time': 0.191032596}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--5b7f5a28-106f-4b78-997c-942e94d8afef-0', usage_metadata={'input_tokens': 11, 'output_tokens': 104, 'total_tokens': 115})}])
content='Please provide me with a question or topic so I ca